<a href="https://colab.research.google.com/github/0yash4/Google-Colab/blob/main/Decision_Tree_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees
## CART - Classification and Regression Trees
### Entropy =
### Gini Impurity =
### Information Gain =

In [1]:
import pandas as pd
import numpy as np

In [51]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

class DecisionTreeClassifier:
  def __init__(self, min_sample_split=2, max_depth=100, rootNode=None):
    self.min_sample_split = min_sample_split
    self.max_depth = max_depth
    self.rootNode = rootNode

  def check_purity(self, y):
    if len(np.unique(y)) == 1:
      return True
    else:
      return False

  def calculateGiniImpurity(self, y):
    values, counts = np.unique(y, return_counts=True)
    probability = counts / len(y)
    giniImpurity = 1 - np.sum(probability**2)
    return giniImpurity

  def calculateEntropy(self, y):
    values, counts = np.unique(y, return_counts=True)
    probability = counts / len(y)
    entropy = -np.sum(probability * np.log2(probability))
    return entropy

  def calculateInformationGain(self, X, y):
    parentEntropy = self.calculateEntropy(y)
    values, counts = np.unique(X, return_counts=True)
    info_gain = 0
    num_bins = len(values)
    bins = [(values[i] + values[i-1])/2 for i in range(1, num_bins)]
    weightedEntropy = float('inf')
    for j in range(1, len(bins)):
      y_left = y[X <= bins[j]]
      y_right = y[X > bins[j]]
      entropy_left = self.calculateEntropy(y_left)
      entropy_right = self.calculateEntropy(y_right)
      weighted_entropy_temp = (len(y_left) / len(y)) * entropy_left + (len(y_right) / len(y)) * entropy_right
      if weighted_entropy_temp < weightedEntropy:
        weightedEntropy = weighted_entropy_temp
      info_gain = parentEntropy - weightedEntropy
    return info_gain

  def best_split(self, X, y):
   x_columns_info_gain: list[int] = []
   for columns in range(X.shape[1]):
    x_columns_info_gain.append(self.calculateInformationGain(X.iloc[:, columns], y))
   best_feature_index = np.argmax(x_columns_info_gain)
   best_feature_name = X.columns[best_feature_index]
   return best_feature_name, best_feature_index

  def best_threshold(self, X, y):
    values, counts = np.unique(X, return_counts=True)
    parent_entropy = self.calculateEntropy(y)
    weighted_entropy_final = []
    Bins = [values[i] + values[i-1]/2 for i in range(1, len(values))]
    for i in Bins:
      y_left = y[X <= i]
      y_right = y[X > i]
      weighted_entropy_temp = (len(y_left) / len(y)) * self.calculateEntropy(y_left) + (len(y_right) / len(y)) * self.calculateEntropy(y_right)
      weighted_entropy = weighted_entropy_temp
      weighted_entropy_final.append(weighted_entropy)
    best_threshold = Bins[np.argmin(weighted_entropy_final)]
    return best_threshold

  def _grow_tree(self, X, y, depth=0, min_samples_split=2):
    n_samples, n_features = X.shape
    n_labels = len(np.unique(y))

    # Stopping criteria
    if (depth >= self.max_depth or
        n_samples < self.min_sample_split or
        self.check_purity(y)):
        print("Shape of y:", y.shape)  # Add this line for debugging
        leaf_value = np.argmax(np.bincount(y.values.flatten()))
        return Node(value=leaf_value)

    # Find best split
    best_feature_name, best_feature_index = self.best_split(X, y)
    # Find best threshold
    best_threshold = self.best_threshold(X.iloc[:, best_feature_index], y)

    # Create child nodes
    left_indices = X[X[best_feature_name] <= best_threshold]
    right_indices = X[X[best_feature_name] > best_threshold]
    left_y = y[X[best_feature_name] <= best_threshold]
    right_y = y[X[best_feature_name] > best_threshold]
    left = self._grow_tree(left_indices, left_y, depth+1)
    right = self._grow_tree(right_indices, right_y, depth+1)
    return Node(best_feature_index, best_threshold, left, right)

  def fit(self, X, y):
    self.root = self._grow_tree(X, y)
    return self.rootNode

  def _predict_single(self, x, node):
        if node.value is not None:
            return node.value

        if x[node.feature] <= node.threshold:
            return self._predict_single(x, node.left)
        else:
            return self._predict_single(x, node.right)

  def predict(self, X):
       return np.array([self._predict_single(x, self.root) for _, x in X.iterrows()])

In [52]:
X = np.random.randint(4, size=(100, 5))
y = np.random.randint(2, size=(100, 1))
X = pd.DataFrame(x, columns=['feature1', 'feature2', 'feature3', 'feature4', 'feature5'])
y = pd.DataFrame(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [60]:
dt = DecisionTreeClassifier(max_depth=4)
dt.fit(X_train, y_train)

Shape of y: (14, 1)
Shape of y: (8, 1)
Shape of y: (3, 1)
Shape of y: (3, 1)
Shape of y: (2, 1)
Shape of y: (2, 1)
Shape of y: (11, 1)
Shape of y: (8, 1)
Shape of y: (2, 1)
Shape of y: (3, 1)
Shape of y: (8, 1)
Shape of y: (3, 1)


In [66]:
y_pred = dt.predict(X_test)

In [69]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average='macro')

0.37969924812030076

In [70]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

In [71]:
y_pred1 = clf.predict(X_test)

In [73]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred1, average='weighted')

0.43403964456596034